In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option("display.max_rows",1300)
pd.set_option("display.max_columns",12000)
pd.set_option('display.max_rows',200000)
pd.set_option('display.width', 320)
team_info = pd.read_csv('team_info.csv')
nhl = pd.read_csv('nhl_sorted_by_dates.csv')
odds = pd.read_csv('odds_all_years.csv')
odds = odds[::2]
nhl2 = nhl.copy()
string = '2018-09-30'
class Season():
    def __init__(self, df, date='0'):
        self.nhl = df
        self.date = date
        df2 = nhl.copy()
        self.s = df2
        self.df_last = df2[-1:] # get last row of data frame
        self.end_reg_season={'2011': 13, '2012': 11, '2013': 30, '2014': 16, '2015': 15, '2016': 13, '2017': 12, '2018': 11, '2019': 10}
        arr = str(self.df_last.date_time).split('-') # get date of last row of df and split to get year
        _ = arr[0].split(" ") # turn series object into something useable
        year = _[-1]
        self.final_year = int(year)
    def last_reg_date(self,reg=0):    #finish to include playoffs
        # split out just the year from date given to class
        splt = self.date.split('-')
        year = int(splt[0])
        month = splt[1]
        if month >='01' and month <='04':
            return str(year)+'-04-'+str(self.end_reg_season.get(str(year))-1)
        if year >= self.final_year:                                 # if year is the end of the data frame return the last date available
            return self.df_last.date_time                           #does not work!!!!!!!!!!!!!!
        else:                                                       # else the year is older than the last year in the df return the end of the season date
            new_year=str(year+1)
            next = new_year+'-04-'+str(self.end_reg_season.get(str(new_year))-1)
            return next
    def get_team_ids(self):
        df = self.nhl
        season = df[(df['date_time'] > self.date) & (df['date_time'] < self.last_reg_date())]
        ids = season.home_team_id.unique()
        return ids
    def season_team(self,team_id,stop='0'):
        df = self.nhl
        if stop=='0':
            season = df[(df['date_time'] > self.date) & (df['date_time'] < self.last_reg_date())]
            season = season[(season['home_team_id']==team_id) | (season['away_team_id']==team_id)]
            self.s = season
            return season
        else:
            season = df[(df['date_time'] > self.date) & (df['date_time'] < stop)]
            season = season[(season['home_team_id'] == team_id) | (season['away_team_id'] == team_id)]
            self.s = season
            return season
    def win_lose(self, away_team, home_team, away_goals, home_goals, team_id):
        if team_id != away_team and team_id != home_team:
            return "error"
        if (away_team == team_id and away_goals > home_goals) or (home_team == team_id and home_goals > away_goals):
            return 'w'
        else:
            return 'l'
    def create_win_column(self,team_id):
        df = self.s
        df['win_loss'] = df.apply(lambda col: self.win_lose(int(col[4]), int(col[5]), int(col[6]), int(col[7]), int(team_id)), axis=1)
        return df
    def team_streak(self,df_form=0):
        lst = self.s.win_loss.tolist()
        ws = [0]
        for i in lst:
            if i == 'w':
                if ws[-1] > 0:
                    ws.append(ws[-1] + 1)
                else:
                    ws.append(1)
            elif i == 'l' and ws[-1] > 0:
                ws.append(-1)
            else:
                ws.append(ws[-1] - 1)
        ws.remove(ws[0])
        if df_form==0:
            return ws
        else:
            self.s['streak'] = ws
            return self.s
    def get_points(dic):
    team_points = {}
    for i in dic:
        lst = dic.get(i)
        points = []
        for j in range(len(lst)):
            if j == 0:
                if lst[j]>0:
                    points.append(2)
                else:
                    points.append(0)
            else:
                if lst[j]>0:
                    points.append(points[j-1]+2)
                else:
                    points.append(points[j-1])
        team_points.update({i:points})
    return team_points

del nhl['date_time_GMT']
del nhl['home_rink_side_start']
del nhl['venue_link']
del nhl['venue_time_zone_id']
yr = '2010'
start = '-09-30'

# create teams dictionary --either of season or streak ...
def data_form(df,yr,start,stop_yr=2019,default=1,tm_id=0,df_or_dict=0):
    tms = {}
    nhl = df
    yr1 = int(yr)
    if default != 0:
        for i in range(stop_yr-yr1):
            season = Season(nhl,(yr + start))
            end = season.end_reg_season.get(str(int(yr)+1))
            for j in season.get_team_ids():
                dt = str(int(yr)+1)+'-04-'+str(end)
                season.season_team(j, stop=dt)
                season.create_win_column(j)
                name = team_info[(team_info['team_id']==j)].abbreviation.max()
                if df_or_dict == 0:
                    tms.update({name +'-'+ yr :season.team_streak()})
                else:
                    tms.update({name +'-'+ yr :season.team_streak(df_form=1)})
            yr = str(int(yr)+1)
    else:
        season = Season(nhl, (yr + start))
#         for j in season.get_team_ids():
        for i in range(stop_yr-yr1):
            end = season.end_reg_season.get(str(int(yr) + 1))
            dt = str(int(yr) + 1) + '-04-' + str(end)
            season.season_team(team)
            season.create_win_column(tm_)
            name = team_info[(team_info['team_id'] == j)].abbreviation.max()
            if df_or_dict == 0:
                tms.update({name + '-' + yr: season.team_streak()})
            else:
                tms.update({name + '-' + yr: season.team_streak(df_form=1)})
            yr = str(int(yr) + 1)
    return tms
def win_streak_per_team_per_season(dct,st,tm_pts,opp_pts):
    overall = []
    st=int(st)
    opp_points_per_team_per_season = {}
    w=int()
    if st>0:
        w=1
    else:
        w=0
    if w ==1:
#  ******* winning streaks ********
        winstreaks_per_team_per_season = {}
        for i in dct:
            lst = []
            tm_opps = []
            strk = dct.get(i)
            tm = tm_pts.get(i)
            op = opp_pts.get(i)
            for j in range(len(strk)):
                if j != len(strk)-1:
                    if strk[j] >= st and strk[j+1]!=strk[j]+1:
                        lst.append(strk[j])
                        overall.append(strk[j])
                        tm_opps.append([tm[j+1],op[j+1],strk[j]])
                else:
                    if strk[j]>= st and strk[j-1]==strk[j]-1:
                        lst.append(strk[j])
                        overall.append(strk[j])
            winstreaks_per_team_per_season.update({i:lst})
            opp_points_per_team_per_season.update({i:tm_opps})
        return winstreaks_per_team_per_season,opp_points_per_team_per_season, overall
    else:
# ******* losing streaks ********
        losingstreaks_per_team_per_season = {}
        for i in dct:
            lst = []
            tm_opps = []
            strk = dct.get(i)
            op = opp_pts.get(i)
            tm = tm_pts.get(i)
            for j in range(len(strk)):
                if j != len(strk)-1:
                    if strk[j] <= st and strk[j+1]!=strk[j]-1:
                        lst.append(strk[j])
                        overall.append(strk[j])
                        tm_opps.append([tm[j+1],op[j+1],strk[j]])
                else:
                    if strk[j]<= st and strk[j-1]==strk[j]+1:
                        lst.append(strk[j])
                        overall.append(strk[j])
            losingstreaks_per_team_per_season.update({i:lst})
            opp_points_per_team_per_season.update({i:tm_opps})
        return losingstreaks_per_team_per_season, opp_points_per_team_per_season, overall
# ***************************************************************

teams = data_form(nhl,yr,start,df_or_dict=1)
teams2 = data_form(nhl,yr,start,df_or_dict=0)
# default = 0
def get_points(dic):
    team_points = {}
    for i in dic:
        lst = dic.get(i)
        points = []
        for j in range(len(lst)):
            if j == 0:
                if lst[j]>0:
                    points.append(2)
                else:
                    points.append(0)
            else:
                if lst[j]>0:
                    points.append(points[j-1]+2)
                else:
                    points.append(points[j-1])
        team_points.update({i:points})
    return team_points

# # make more dictionaries instead of lambda functions
# # ---opponent name,streak,points dict, game_id dict, date_dict?

team_id_dict = {}
ids = team_info.team_id.tolist()
names = team_info.abbreviation.tolist()
for i in range(len(ids)):
    team_id_dict.update({ids[i]:names[i]})
names_id_dict = {}
for i in range(len(names)):
    names_id_dict.update({names[i]:ids[i]})
all_yrs_tms_dic = {}
for na in names_id_dict.keys():
    df8787 = pd.DataFrame()
    for i in teams87:
        name_year = i.split('-')
        if name_year[0]==na:
            df8787 = df8787.append(teams87.get(i))
    all_yrs_tms_dic.update({na:df8787})
def names(id):  #add team_info df?
    return team_info[(team_info['team_id']==id)].abbreviation.max()
def opp_points(date,name,away,home): #add tm_pts dict and or teams_df?
    tm_yr = name.split("-")
    name = tm_yr[0]
    if away==name:
        tm = home+'-'+tm_yr[1]
        df = teams.get(tm)
        return df[(df['date_time']==date)].points.max()
    else:
        tm = away+'-'+tm_yr[1]
        df = teams.get(tm)
        return df[(df['date_time'] == date)].points.max()
# def opp_streak
tm_pts = get_points(teams2)
opp_points_dict = {}
# all_team_years_df = pd.DataFrame()
for i in teams:
    team = teams.get(i)
    y = teams2.get(i)
    y = tm_pts.get(i)
    team['points']=y
    team['name']=i
for i in teams:
    team= teams.get(i)
    team['away_team']=team.apply(lambda col: names(int(col[4])),axis=1)
    team['home_team']=team.apply(lambda col: names(int(col[5])),axis=1)
    team['opp_points'] = team.apply(lambda col: opp_points(col[3], col[13],col[14],col[15]),axis=1)
    opp_points_dict.update({i:team['opp_points'].tolist()})
#     all_team_years_df.append(teams.get(i))
# all_team_years_df.to_csv('all_team_years.csv',encoding='utf-8')
    # team['opp_streak'] = team.apply(lambda col: opp_streak(col[3],col[13],col[14],col[15]),axis=1)
# print(teams.get('MIN-2010'))


# print(team_id_dict)


# print(y)

#
# x['home_name'] = team_info[(team_info['team_id']==x['home_team_id'])].abbreviation.max()


dct,pts,lst = win_streak_per_team_per_season(teams2,4,tm_pts,opp_points_dict)
diff_significant={}
diff=[]
for i in pts:
    team_year = pts.get(i)
    for j in team_year:
        x = j[0]-j[1]
        diff.append(x)
# *****************************************************************************************************


In [ ]:
# names_id_dict.get()
# def get_team(name,yr = '2010',stop_yr=2019):
# name='COL'
# tm_id57=names_id_dict.get(name)
# yr = '2010'
# stop_yr=2019
# df = pd.DataFrame()
# for i in range(stop_yr-int(yr)):
#     season = Season(nhl,yr+'-09-30')
#     x = season.season_team(tm_id57)
#     df = df.append(x)
#     yr = str(int(yr)+1)
# df



# def cum_point_diff(s,filename):
#     strk_len = s
#     dct,pts,lst = win_streak_per_team_per_season(teams2,strk_len,tm_pts,opp_points_dict)
#     strk_lens,cnts = get_counts_of_streaks(strk_len,lst)
#     years1 = {}
#     yr1 = int(yr)
#     for i in range(9):
#         years1.update({yr1:{}})
#         yr1+=1
#     yr_num_teams = {}
#     temp_yr = yr
#     # seasons = {}
#     for i in range(9):
#         season2 = Season(nhl,(temp_yr+start))
#     #     seasons.update({years[i]:season2)
#         num = len(season2.get_team_ids())
#         yr_num_teams.update({temp_yr:num})
#         temp_yr = str(int(temp_yr)+1)
#     pts2 = get_point_diff_dict(pts)  #pts 2=flattened pts
#     years = {}
#     for year in yr_num_teams:
#         dic = {}
#         for tm_yr in pts2:
#             lst = tm_yr.split("-")
#             if lst[1] == year:
#                 dic.update({tm_yr:(pts2.get(tm_yr))})
#         years.update({year:dic})
#     year_plot_prep = []
#     for year in years:
#         d = get_point_diffs_of_streaks2(strk_lens,years.get(year))
#         data = pd.DataFrame.from_dict({'Streak_Lengths':d.keys(),'Cumulative_Point_Differential':d.values()},orient='index')
#         data = data.T
#         year_plot_prep.append(data)
        
        
        
        
# # def teamplots(s,filename):
# dic57 = {}
# for name in names_id_dict:
# # name= 'COL'
#     tm_id57=names_id_dict.get(name)
#     yr = '2010'
#     stop_yr=2019
#     for i in range(stop_yr-int(yr)):
#         season = Season(nhl,yr+'-09-30')
#         df = season.season_team(tm_id57)
        
#         yr = str(int(yr)+1)          
#     dic57.update({name:df})







# df = get_team('COL')
# print(df)
# teams4 = data_form(nhl,'2010',start,stop_yr=2012,df_or_dict=0)
# # teams = data_form(nhl,yr,start,df_or_dict=1)
# # teams2 = data_form(nhl,yr,start,df_or_dict=0)
# # default = 0
# def get_points(dic):
#     team_points = {}
#     for i in dic:
#         lst = dic.get(i)
#         points = []
#         for j in range(len(lst)):
#             if j == 0:
#                 if lst[j]>0:
#                     points.append(2)
#                 else:
#                     points.append(0)
#             else:
#                 if lst[j]>0:
#                     points.append(points[j-1]+2)
#                 else:
#                     points.append(points[j-1])
#         team_points.update({i:points})
#     return team_points
# # team_id_dict = {}
# # ids = team_info.team_id.tolist()
# # names = team_info.abbreviation.tolist()
# # for i in range(len(ids)):
# #     team_id_dict.update({ids[i]:names[i]})
# # def names(id):  #add team_info df?
# #     return team_info[(team_info['team_id']==id)].abbreviation.max()
# def opp_points(date,name,away,home): #add tm_pts dict and or teams_df?
#     tm_yr = name.split("-")
#     name = tm_yr[0]
#     if away==name:
#         tm = home+'-'+tm_yr[1]
#         df = teams4.get(tm)
#         return df[(df['date_time']==date)].points.max()
#     else:
#         tm = away+'-'+tm_yr[1]
#         df = teams4.get(tm)
#         return int(df[(df['date_time'] == date)].points.max())
# # def opp_streak
# tm_pts2 = get_points(teams4)
# opp_points_dict2 = {}
# # all_team_years_df = pd.DataFrame()
# # team = pd.DataFrame()
# # for i in teams3:
# team = teams3.get('MIN-2010')
# #     y = tm_pts2.get(i)
# #     team['points']=y
# #     team['name']=i
# # team
# # for i in teams3:
# #     team= teams3.get(i)
# #     team['away_team']=team.apply(lambda col: names(int(col[4])),axis=1)
# #     team['home_team']=team.apply(lambda col: names(int(col[5])),axis=1)
# #     team['opp_points'] = team.apply(lambda col: opp_points(col[3], col[13],col[14],col[15]),axis=1)
# #     opp_points_dict2.update({i:team['opp_points'].tolist()})
# # for i in teams:
# #     team = teams.get(i)
# #     y = teams2.get(i)
# #     y = tm_pts.get(i)
# #     team['points']=y
# #     team['name']=i
# # for i in teams:
# #     team= teams.get(i)
# #     team['away_team']=team.apply(lambda col: names(int(col[4])),axis=1)
# #     team['home_team']=team.apply(lambda col: names(int(col[5])),axis=1)
# #     team['opp_points'] = team.apply(lambda col: opp_points(col[3], col[13],col[14],col[15]),axis=1)
# #     opp_points_dict.update({i:team['opp_points'].tolist()})
# #     all_team_years_df.append(teams.get(i))
# dct57,pts57,lst57 = win_streak_per_team_per_season(teams3,4,tm_pts2,opp_points_dict2)
# # diff_significant={}
# # diff=[]
# # for i in pts:
# #     team_year = pts.get(i)
# #     for j in team_year:
# #         x = j[0]-j[1]
# #         diff.append(x)

# # teams = data_form(nhl,yr,start,df_or_dict=1)
# # teams2 = data_form(nhl,yr,start,df_or_dict=0)
#

teams87 = data_form(nhl,yr,start,df_or_dict =1)
teams87

In [ ]:
all_yrs_tms_dic = {}
for na in names_id_dict.keys():
    df8787 = pd.DataFrame()
    for i in teams87:
        name_year = i.split('-')
        if name_year[0]==na:
            df8787 = df8787.append(teams87.get(i))
    all_yrs_tms_dic.update({na:df8787})
df89= all_yrs_tms_dic.get('MIN')
df89

In [ ]:
len(pts)

In [ ]:
len(dct)

In [ ]:
len(lst)

In [ ]:
def get_counts_of_streaks(streak_length,lst):
    st_lens=[]
    cnts=[]
    if streak_length<0:
        for i in range(streak_length,min(lst)-1,-1):
            st_lens.append(i)
            cnts.append(lst.count(i))
    else:
        for i in range(streak_length,max(lst)+1):
            st_lens.append(i)
            cnts.append(lst.count(i))
    return st_lens,cnts

In [ ]:
def point_diff_avg():
    for i in pts:
        team_year = pts.get(i)
        for j in team_year:
            if len(j)!=0:
                x = j[0]-j[1]
                diff.append(x)
    return sum(diff)/len(diff)
def get_point_diff_dict(pts):
    pts2={}
    for i in pts:
        diff=[]
        team_year = pts.get(i)
        for j in team_year:
            x = j[0]-j[1]
            diff.append([x,j[2]])
        pts2.update({i:diff})
    return pts2

In [ ]:
pts

In [ ]:
# years = []
# yr1 = int(yr)
# for i in range(9):
#     years.append(yr1)
#     yr1+=1
# years
# yr2 = str(yr)
# stop_yr= int(yr)+1
# stop_yr
# teams1 = data_form(nhl,yr2,start,stop_yr)
# tm_pts1 =get_points(teams1)
# tm_pts1
# teams3 = data_form(nhl,yr2,start,stop_yr,df_or_dict=1)
# tm_pts1
strk_len = -3
dct,pts,lst = win_streak_per_team_per_season(teams2,strk_len,tm_pts,opp_points_dict)
strk_lens,cnts = get_counts_of_streaks(strk_len,lst)

In [ ]:

# diff = []
# strk_len = -4
# dct,pts,lst = win_streak_per_team_per_season(teams2,strk_len,tm_pts,opp_points_dict)
# point_diff_avg()
# str_lens, counts = get_counts_of_streaks(strk_len,lst)
# pts
years1 = {}
yr1 = int(yr)
for i in range(9):
    years1.update({yr1:{}})
    yr1+=1
yr_num_teams = {}
temp_yr = yr
# seasons = {}
for i in range(9):
    season2 = Season(nhl,(temp_yr+start))
#     seasons.update({years[i]:season2)
    num = len(season2.get_team_ids())
    yr_num_teams.update({temp_yr:num})
    temp_yr = str(int(temp_yr)+1)
pts2 = get_point_diff_dict(pts)  #pts 2=flattened pts
years = {}
for year in yr_num_teams:
    dic = {}
    for tm_yr in pts2:
        lst = tm_yr.split("-")
        if lst[1] == year:
            dic.update({tm_yr:(pts2.get(tm_yr))})
    years.update({year:dic})


In [ ]:

# def get_point_diffs_of_streaks(sls,pt):
#     d = {}
#     for i in sls:
#         d.update({i:0})
#     for j in pt:
#         team_year = pt.get(j)
#         for x in team_year:
#             if len(x)!=0:
#                 dif = x[0]-x[1]
#                 d.update({x[2]:(d.get(x[2])+dif)})
#     return d
# alpha = years.get('2015')
# d = {}
# for i in strk_lens:
#     d.update({i:0})
# for j in alpha:
#     team_year = alpha.get(j)
#     for x in team_year:
#         if len(x)!=0:
#             dif = x[0]
#             d.update({x[1]:(d.get(x[1])+dif)})
# d
def years_tolist(some_years):
    lst57 =[]
    for i in some_years.keys():
        lst57.append(i)
    return lst57
def get_point_diffs_of_streaks2(sls,pt):
    d = {}
    for i in sls:
        d.update({i:0})
    for i in pt:
        team_year = pt.get(i)
        for x in team_year:
            if len(x)!=0:
                dif = x[0]
                d.update({x[1]:(d.get(x[1])+dif)})
    return d

# d = get_point_diffs_of_streaks2(strk_lens,years.get('2010'))
def cum_pt_diff(s,tile,filename):
    strk_len = s
    dct,pts,lst = win_streak_per_team_per_season(teams2,strk_len,tm_pts,opp_points_dict)
    strk_lens,cnts = get_counts_of_streaks(strk_len,lst)
    years1 = {}
    yr1 = int(yr)
    for i in range(9):
        years1.update({yr1:{}})
        yr1+=1
    yr_num_teams = {}
    temp_yr = yr
    # seasons = {}
    for i in range(9):
        season2 = Season(nhl,(temp_yr+start))
    #     seasons.update({years[i]:season2)
        num = len(season2.get_team_ids())
        yr_num_teams.update({temp_yr:num})
        temp_yr = str(int(temp_yr)+1)
    pts2 = get_point_diff_dict(pts)  #pts 2=flattened pts
    years = {}
    for year in yr_num_teams:
        dic = {}
        for tm_yr in pts2:
            lst = tm_yr.split("-")
            if lst[1] == year:
                dic.update({tm_yr:(pts2.get(tm_yr))})
        years.update({year:dic})
    year_plot_prep = []
    for year in years:
        d = get_point_diffs_of_streaks2(strk_lens,years.get(year))
        data = pd.DataFrame.from_dict({'Streak_Lengths':d.keys(),'Cumulative_Point_Differential':d.values()},orient='index')
        data = data.T
        year_plot_prep.append(data)

# data.plot(kind='barh',color=(data['Cumulative_Point_Differential']>0).map({True:'g',False:'r'}))
# fig, ax = plt.subplots(2,5,figsize=(8,8))


    fig = plt.figure(figsize=(20,8))
    # fig.text
    # plt.style.use('ggplot')
    yrs = years_tolist(years)
    for i in range(len(year_plot_prep)):
        data = year_plot_prep[i]
        barheights = data.Cumulative_Point_Differential
        barlabels = data.Streak_Lengths
        ax = fig.add_subplot(2,5,i+1)
        ax.bar(range(len(barheights)), barheights, color=(barheights>0).map({True: 'black',False: 'r'}))
        ax.set_xticks(range(len(barheights)))
        ax.set_xticklabels(barlabels,rotation=45,fontsize=10)
        ax.set_ylabel('Cumulative_Point_Differential',fontsize=14)
        ax.set_xlabel('Streak_Lengths',fontsize=14)
    #     ax.text(x=.01,y=.01,s=yrs[i],color='g',ha='center',va='top',fontsize=17)
        ax.set_title(yrs[i])
    fig.tight_layout(pad=3)
    fig.suptitle(title,fontsize=15)
    fig.savefig(filename,dpi=125)
# barheights = data.Cumulative_Point_Differential
# barlabels = data.Streak_Lengths
# ax.bar(range(len(barheights)), barheights, color=(barheights>0).map({True: 'black',False: 'r'}))
# ax.set_xticks(range(len(barheights)))
# ax.set_xticklabels(barlabels)
# ax.set_ylabel('Cumulative_Point_Differential')
# ax.set_xlabel('Streak_Lengths')
# ax.set_title('2010')
# x = d.keys()
# y = d.values()
# y
cum_pt_diff(-3,'Each Season of Cumulitive Point Differentials Between Teams Ending a Losing Streak and Their Oppontents','Cum_losing_streak_years.png')
cum_pt_diff(4,'Each Season of Cumulitive Point Differentials Between Teams Ending a Winning Streak and Their Oppontents''Cum_winning_streak_years.png')


In [ ]:
def season_streaks_plot(s,ye,filename):
#     strk_len = s
#     dct,pts,lst = win_streak_per_team_per_season(teams2,strk_len,tm_pts,opp_points_dict)
#     strk_lens,cnts = get_counts_of_streaks(strk_len,lst)
#     years1 = {}
#     yr1 = int(yr)
#     for i in range(9):
#         years1.update({yr1:{}})
#         yr1+=1
#     yr_num_teams = {}
#     temp_yr = yr
    # seasons = {}
#     for i in range(9):
#         season2 = Season(nhl,(temp_yr+start))
#     #     seasons.update({years[i]:season2)
#         num = len(season2.get_team_ids())
#         yr_num_teams.update({temp_yr:num})
#         temp_yr = str(int(temp_yr)+1)
#     pts2 = get_point_diff_dict(pts)  #pts 2=flattened pts
#     years = {}
#     for year in yr_num_teams:
#         dic = {}
#         for tm_yr in pts2:
#             lst = tm_yr.split("-")
#             if lst[1] == year:
#                 dic.update({tm_yr:(pts2.get(tm_yr))})
#         years.update({year:dic})
#     for year in years:
#         d = get_point_diffs_of_streaks2(strk_lens,years.get(year))
#         data = pd.DataFrame.from_dict({'Streak_Lengths':d.keys(),'Cumulative_Point_Differential':d.values()},orient='index')
#         data = data.T
#         year_plot_prep.append(data)
    yr_lst = years_tolist(years)
    year_plot_prep = []
    for year in yr_lst: 
        d = (counts_of_strk_per_season(s,year))
        data = pd.DataFrame.from_dict({'Streak_Lengths':d.keys(),'Total_Number_of_Each_Streak':d.values()},orient='index')
        data = data.T
        year_plot_prep.append(data)
    fig = plt.figure(figsize=(20,8))
    # fig.text
    # plt.style.use('ggplot')
    yrs = years_tolist(years)
    for i in range(len(year_plot_prep)):
        data = year_plot_prep[i]
        barheights = data.Total_Number_of_Each_Streak
        barlabels = data.Streak_Lengths
        ax = fig.add_subplot(2,5,i+1)
        ax.bar(range(len(barheights)), barheights, color=(barheights>0).map({True: 'g',False: 'r'}))
        ax.set_xticks(range(len(barheights)))
        ax.set_xticklabels(barlabels,rotation=45,fontsize=10)
        ax.set_ylabel('Total Number of Each Streak',fontsize=14)
        ax.set_xlabel('Streak_Lengths',fontsize=14)
    #     ax.text(x=.01,y=.01,s=yrs[i],color='g',ha='center',va='top',fontsize=17)
        ax.set_title(yrs[i])
    fig.tight_layout(pad=3)
    fig.suptitle('Each Season Total Number of Each Streak Achived by All Teams Combined',fontsize=15)
    fig.savefig(filename,dpi=125)
# # barheights = data.Cumulative_Point_Differential
# # barlabels = data.Streak_Lengths
# # ax.bar(range(len(barheights)), barheights, color=(barheights>0).map({True: 'black',False: 'r'}))
# # ax.set_xticks(range(len(barheights)))
# # ax.set_xticklabels(barlabels)
# # ax.set_ylabel('Cumulative_Point_Differential')
# # ax.set_xlabel('Streak_Lengths')
# # ax.set_title('2010')
# # x = d.keys()
# # y = d.values()
# # y
season_streaks_plot(-3,'2010','something_else.png')
season_streaks_plot(4,'2010','Cum_winning_streak_years87.png')

In [ ]:
def counts_of_strk_per_season(st,year):
    st_len =[]
    cnts = []
    yr1 = int(year)+1
    ts = data_form(nhl,year,start,yr1)
    dct,_,lst = win_streak_per_team_per_season(ts,st,tm_pts,opp_points_dict)
    dic={}
    if st<0:
        for i in range(st,min(lst)-1,-1):
#             st_len.append(i)
#             cnts.append(lst.count(i))
            dic.update({i:lst.count(i)})
    else:
        for i in range(st,max(lst)+1):
#             st_len.append(i)
#             cnts.append(lst.count(i))
            dic.update({i:lst.count(i)})
    return dic
dics = {}
yr_lst = years_tolist(years)
for i in yr_lst:
    d = counts_of_strk_per_season(4,i)
    dics.update({i:d})
dics
# for i in range(strk_len,min(lst)-1,-1):
#     st_len.append(i)
#     cnts.append(lst.count(i))

# plt.bar(st_len,cnts)
# plt.show()

In [ ]:

def sort_tuple(tup):  
    # getting length of list of tuples 
    lst = len(tup)  
    for i in range(0, lst):  
        for j in range(0, lst-i-1):  
            if (tup[j][1] > tup[j + 1][1]):  
                temp = tup[j]  
                tup[j]= tup[j + 1]  
                tup[j + 1]= temp  
    return tup  
def single_season_streak_breaks(s,filename):
    strk_len = s
    dct,pts,lst = win_streak_per_team_per_season(teams2,strk_len,tm_pts,opp_points_dict)
    strk_lens,cnts = get_counts_of_streaks(strk_len,lst)
    years1 = {}
    yr1 = int(yr)
    for i in range(9):
        years1.update({yr1:{}})
        yr1+=1
    yr_num_teams = {}
    temp_yr = yr
    # seasons = {}
    for i in range(9):
        season2 = Season(nhl,(temp_yr+start))
    #     seasons.update({years[i]:season2)
        num = len(season2.get_team_ids())
        yr_num_teams.update({temp_yr:num})
        temp_yr = str(int(temp_yr)+1)
    pts2 = get_point_diff_dict(pts)  #pts 2=flattened pts
    years = {}

    for year in yr_num_teams:
        dic = {}
        for tm_yr in pts2:
            lst = tm_yr.split("-")
            if lst[1] == year:
                dic.update({tm_yr:(pts2.get(tm_yr))})
        years.update({year:dic})

    year = years.get('2015')
    games =[]
    which_strk= []
    pt_diff = []



    nvals= []
    names = []
    games ={}
    for i in year:
        nma = year.get(i)

        for j in nma:
            if len(year.get(i))>0:
                games.update({i:year.get(i)})
                which_strk.append(j[1])
                nvals.append(j[0])
    # names=sorted(names)
    # games=pd.DataFrame.from_dict(games)
    #         names.append(i)
    #         nm = []
    #         for j in year.get(i):
    #             nm.append(j[1])
    #             which_strk.append(j[0])
    #         nvals.append(nm)

    #     year_plot_prep = []
    #     for year in years:
    #         d = get_point_diffs_of_streaks2(strk_lens,years.get(year))
    #         data = pd.DataFrame.from_dict({'Streak_Lengths':d.keys(),'Cumulative_Point_Differential':d.values()},orient='index')
    #         data = data.T
    #         year_plot_prep.append(data)
    # year_plot_prep = []
    # for i in games:
    #     for j in games.get(i):
    #         data = pd.DataFrame.from_dict({'Streak_Lengths':j[1],'Differential_By_Game':j[0]},orient='index')
    # #         data = data.T
    #         year_plot_prep.append(data)
    # year_plot_prep
    TOR = nvals[0]
    BUF = nvals[1]
    BOS = nvals[2]
    CGY = nvals[3]
    NSH = nvals[4]
    STL = nvals[5]
    CBJ = nvals[6]
    CAR = nvals[7]
    FLA = nvals[8]
    VAN = nvals[9]
    SJS = nvals[10]
    ARI = nvals[11]
    MIN = nvals[12]
    PHI = nvals[13]
    PIT = nvals[14]
    ANA = nvals[15]
    MTL = nvals[16]
    EDM = nvals[17]
    WPG = nvals[18]
    plt.style.use('ggplot')
    fig, ax = plt.subplots(figsize=(10,10))
    barheights = nvals
    barlabels = which_strk
    ax.bar(range(len(barheights)), barheights,color=('maroon','b','purple','darkorange','g','k'))
    ax.set_xticks(range(len(barheights)))
    ax.set_xticklabels(barlabels, rotation=45)
    fig.tight_layout()
    plt.show()
    # fig, ax = plt.subplots(figsize=(8,8))
    # ax.hist(year_plot_prep)
    # # for i in range(len(nvals)):
    # #     pt_diff.update({name[i]:nvals[i]})
    TOR = nvals[0]
    BUF = nvals[1]
    BOS = nvals[2]
    CGY = nvals[3]
    NSH = nvals[4]
    STL = nvals[5]
    CBJ = nvals[6]
    CAR = nvals[7]
    FLA = nvals[8]
    VAN = nvals[9]
    SJS = nvals[10]
    ARI = nvals[11]
    MIN = nvals[12]
    PHI = nvals[13]
    PIT = nvals[14]
    ANA = nvals[15]
    MTL = nvals[16]
    EDM = nvals[17]
    WPG = nvals[18]
    # fig = plt.figure()
    # ax = fig.add_subplot(111)
    # rands = np.random.choice(30,19,replace=False)
    # rects =[]
    # width =.011
    # for i in range(19):
    #     if i == 0:
    #         ind=which_strk[i]
    #         rects.append(ax.bar(ind,nvals[i],width,color=)
    #     else:
    #         ind = which_strk[i]
    #         rects.append(ax.bar(ind+width*i,nvals[i],width,color=int(np.random.random()*100)))

    # ax.set_ylabel('Scores')
    # ax.set_xticks(ind+width)
    # ax.set_xticklabels( ('2011-Jan-4', '2011-Jan-5', '2011-Jan-6') )
    # ax.legend( (rects1[0], rects2[0], rects3[0]), ('y', 'z', 'k') )

    # def autolabel(rects):
    #     for rect in rects:
    #         h = rect.get_height()
    #         ax.text(rect.get_x()+rect.get_width()/2., 1.05*h, '%d'%int(h),
    #                 ha='center', va='bottom')
    # autolabel(rects)
    # # autolabel(rects1)
    # # autolabel(rects2)
    # # autolabel(rects3)

    # plt.show()
    # spc_dic = counts_of_strk_per_season(-3,'2015')
    # year_plot_prep = []
    # d = sorted_games
    # for i in d:
    #     data = pd.DataFrame.from_dict({'Streak_Lengths':i[1],'Differential_By_Game':i[0]},orient='index')
    #     data = data.T
    #     year_plot_prep.append(data)

        # fig.text
    
    # plt.show()
    # yrs = years_tolist(years)
    # for i in year_plot_prep:
    #     data = i
    #     barheights = data.Differential_By_Game
    #     barlabels = data.Streak_Lengths
    #     ax.bar(range(len(barheights)), barheights, color=(barheights>0).map({True: 'black',False: 'r'}))
    # ax.set_xticks(range(len(barheights)))
    # ax.set_xticklabels(barlabels,rotation=45,fontsize=10)
    # ax.set_ylabel('Differential By game',fontsize=14)
    # ax.set_xlabel('Streak_Lengths',fontsize=14)
    # #     ax.set_title(yrs[i])
    # fig.tight_layout(pad=3)
    # fig.suptitle('Each Season of Cumulitive Point Differentials Between Teams Ending a Losing Streak and Their Oppontents',fontsize=15)
    fig.savefig(filename,dpi=125)

single_season_streak_breaks(-5,'2015_End_Of_Losing_Streaks_Point_Diff.png')
single_season_streak_breaks(5,'2015_End_Of_Winning_Streaks_Point_Diff.png')

In [ ]:

teams3 = data_form(nhl,'2010','-09-30',stop_yr=2012,default=0,df_or_dict=0,)
teams3

In [ ]:
print(5)

In [ ]:
def get_opp_points(teams_df,teams_dict,tm_pts):
    opp_points_dict = {}
    for i in teams_df:
        team = teams_df.get(i)
        y = teams_dict.get(i)
        y = tm_pts.get(i)
        team['points']=y
        team['name']=i
    for i in teams_df:
        team= teams_df.get(i)
        team['away_team']=team.apply(lambda col: names(int(col[4])),axis=1)
        team['home_team']=team.apply(lambda col: names(int(col[5])),axis=1)
        team['opp_points'] = team.apply(lambda col: opp_points(col[3], col[13],col[14],col[15]),axis=1)
        opp_points_dict.update({i:team['opp_points'].tolist()})
        all_team_years_df.append(team)
    return opp_points_dict
op_dict = get_opp_points(teams3,teams1,tm_pts1)
dct1,pts1,lst1 = win_streak_per_team_per_season(teams3,4,tm_pts1,op_dict)

In [ ]:
len(lst)
